In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import os,random, math
import gc

In [0]:
!pip3 install --upgrade kaggle
import os
os.environ['KAGGLE_USERNAME'] = "tedghmn" # username from the json file
os.environ['KAGGLE_KEY'] = "9c7742978732a1456c48d4405a4bd31e"


!kaggle competitions download -c ashrae-energy-prediction

Requirement already up-to-date: kaggle in /usr/local/lib/python3.6/dist-packages (1.5.6)
weather_test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
building_metadata.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
weather_train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
#!unzip \*.zip

# **Importing data and Feature engineering**

## **Train.csv**

In [0]:
import pandas as pd
train_df = pd.read_csv('train.csv.zip')
train_df.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [0]:
train_df.shape

(20216100, 4)

## **building_metadata.csv**

In [0]:
building_meta_df = pd.read_csv('building_metadata.csv')
building_meta_df.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [0]:
building_meta_df.site_id.nunique()

16

**Drop columns year_build and floor_count**




In [0]:
building_meta_df = building_meta_df.drop(columns=['year_built','floor_count'], axis=1)

In [0]:
building_meta_df.head()

,site_id,building_id,primary_use,square_feet
0,0,0,Education,7432
1,0,1,Education,2720
2,0,2,Education,5376
3,0,3,Education,23685
4,0,4,Education,116607


In [0]:
print('There are', building_meta_df.primary_use.nunique(),'values for primary_use ' )

There are 16 values for primary_use 


In [0]:
 building_meta_df.primary_use.value_counts()

Education                        549
Office                           279
Entertainment/public assembly    184
Public services                  156
Lodging/residential              147
Other                             25
Healthcare                        23
Parking                           22
Warehouse/storage                 13
Manufacturing/industrial          12
Retail                            11
Services                          10
Technology/science                 6
Food sales and service             5
Utility                            4
Religious worship                  3
Name: primary_use, dtype: int64

In [0]:
#replace categories with low values with 'others'

t=["Religious worship","Utility","Food sales and service","Services","Retail","Manufacturing/industrial","Warehouse/storage","Parking","Healthcare", "Technology/science","Other"]

In [0]:
building_meta_df.loc[building_meta_df.primary_use.isin(t),'primary_use'] = 'others'

In [0]:
#primary_use: categorical into numerical values

building_meta_df["primary_use"] = building_meta_df["primary_use"].astype('category')
building_meta_df["primary_use"] = building_meta_df["primary_use"].cat.codes

In [0]:
building_meta_df['primary_use'] = building_meta_df.primary_use.astype("int64")

In [0]:
building_meta_df.columns

Index(['site_id', 'building_id', 'primary_use', 'square_feet'], dtype='object')

In [0]:
building_meta_df.head()

,site_id,building_id,primary_use,square_feet
0,0,0,0,7432
1,0,1,0,2720
2,0,2,0,5376
3,0,3,0,23685
4,0,4,0,116607


In [0]:
building_meta_df.tail()

,site_id,building_id,primary_use,square_feet
1444,15,1444,1,19619
1445,15,1445,0,4298
1446,15,1446,1,11265
1447,15,1447,2,29775
1448,15,1448,3,92271


In [0]:
weather_train_df = pd.read_csv('weather_train.csv.zip')
weather_train_df.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [0]:
print('Size of weather_train_df data', weather_train_df.shape)


Size of weather_train_df data (139773, 9)


In [0]:
weather_train_df.columns

Index(['site_id', 'timestamp', 'air_temperature', 'cloud_coverage',
       'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure',
       'wind_direction', 'wind_speed'],
      dtype='object')

In [0]:
weather_train_df.isnull().sum()

site_id                   0
timestamp                 0
air_temperature          55
cloud_coverage        69173
dew_temperature         113
precip_depth_1_hr     50289
sea_level_pressure    10618
wind_direction         6268
wind_speed              304
dtype: int64

**Drop columns with high frequency of null values**

In [0]:
weather_train_df = weather_train_df.drop(columns=['precip_depth_1_hr','wind_direction','sea_level_pressure','cloud_coverage'], axis=1)


**Replace null values of**

1.   **air_temperature**
2.   **wind_speed**
1.   **dew_temperature**

**with the mean**










In [0]:
weather_train_df['air_temperature'].fillna(weather_train_df['air_temperature'].mean(), inplace=True)
weather_train_df['wind_speed'].fillna(weather_train_df['wind_speed'].mean(), inplace=True)
weather_train_df['dew_temperature'].fillna(weather_train_df['dew_temperature'].mean(), inplace=True)

In [0]:
weather_train_df.head()

,site_id,timestamp,air_temperature,dew_temperature,wind_speed
0,0,2016-01-01 00:00:00,25.0,20.0,0.0
1,0,2016-01-01 01:00:00,24.4,21.1,1.5
2,0,2016-01-01 02:00:00,22.8,21.1,0.0
3,0,2016-01-01 03:00:00,21.1,20.6,0.0
4,0,2016-01-01 04:00:00,20.0,20.0,2.6


# **Concat dataFrames**

In [0]:
#Merge train_df and building_train_df

myData =  pd.merge(train_df,building_meta_df, on=['building_id'],sort=False)

In [0]:
#Merge train_df and building_train_df and weather_train_df into my_data

myData=pd.merge(myData, weather_train_df, on=['site_id','timestamp'],sort=False)


In [0]:
myData.columns

Index(['building_id', 'meter', 'timestamp', 'meter_reading', 'site_id',
       'primary_use', 'square_feet', 'air_temperature', 'dew_temperature',
       'wind_speed'],
      dtype='object')

In [0]:
myData.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,air_temperature,dew_temperature,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,0,7432,25.0,20.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,0,2720,25.0,20.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,0,5376,25.0,20.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,0,23685,25.0,20.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,0,116607,25.0,20.0,0.0


In [0]:
# Split timestamp into month and day 

myData["month"] = myData["timestamp"].map(lambda row: row.split(" ")[0].split("-")[1])
myData["day"] = myData["timestamp"].map(lambda row: row.split(" ")[0].split("-")[2])



myData['month'] = pd.to_numeric(myData.month, errors='coerce')

myData['day'] = pd.to_numeric(myData.month, errors='coerce')


In [0]:
myData = myData.drop(columns=[ 'timestamp','site_id'], axis=1)

In [0]:
myData.columns

Index(['building_id', 'meter', 'meter_reading', 'primary_use', 'square_feet',
       'air_temperature', 'dew_temperature', 'wind_speed', 'month', 'day'],
      dtype='object')

In [0]:
myData.head()

,building_id,meter,meter_reading,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,month,day
0,0,0,0.0,0,7432,25.0,20.0,0.0,1,1
1,1,0,0.0,0,2720,25.0,20.0,0.0,1,1
2,2,0,0.0,0,5376,25.0,20.0,0.0,1,1
3,3,0,0.0,0,23685,25.0,20.0,0.0,1,1
4,4,0,0.0,0,116607,25.0,20.0,0.0,1,1


# *Split myData into TrainDF and TestDF*

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
trainDF,testdf = train_test_split(myData, test_size=0.2, random_state=45)


In [0]:
trainDF.head()

,building_id,meter,meter_reading,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,month,day
9606115,835,0,43.4167,1,17220,13.9,11.1,4.1,12,12
184779,78,1,1392.9400,0,76257,22.2,16.1,4.1,3,3
10846520,955,2,311.1000,0,198488,33.3,23.9,2.6,6,6
14979167,1139,1,1847.2600,3,336650,20.0,10.6,2.6,9,9
1075753,97,1,1055.2600,2,187647,17.8,6.7,0.0,12,12


In [0]:
testdf.head()

,building_id,meter,meter_reading,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,month,day
12149165,959,1,266.9990,0,128536,12.2,8.3,0.0,12,12
7366675,728,0,5.3000,0,30300,4.0,3.0,8.2,2,2
16841290,1299,2,333.5230,0,25401,24.4,2.8,2.1,5,5
1246907,125,0,134.9000,0,16802,5.1,3.1,4.1,4,4
19061602,1358,1,36.6751,2,61439,20.0,7.2,4.1,6,6


**Split into x_train and y_train**

In [0]:
x_train = trainDF[['meter', 'building_id', 'primary_use', 'month', 'day','air_temperature','dew_temperature', 'wind_speed','square_feet']]
y_train = trainDF['meter_reading']

In [0]:
print('Size of x_train data', x_train.shape)
print('Size of y_train data', y_train.shape)

Size of x_train data (16100484, 9)
Size of y_train data (16100484,)


In [0]:
x_train .head()

,meter,building_id,primary_use,month,day,air_temperature,dew_temperature,wind_speed,square_feet
9606115,0,835,1,12,12,13.9,11.1,4.1,17220
184779,1,78,0,3,3,22.2,16.1,4.1,76257
10846520,2,955,0,6,6,33.3,23.9,2.6,198488
14979167,1,1139,3,9,9,20.0,10.6,2.6,336650
1075753,1,97,2,12,12,17.8,6.7,0.0,187647


In [0]:
y_train .head()

9606115       43.4167
184779      1392.9400
10846520     311.1000
14979167    1847.2600
1075753     1055.2600
Name: meter_reading, dtype: float64

In [0]:
x_test = testdf[['meter', 'building_id', 'primary_use', 'month', 'day','air_temperature','dew_temperature', 'wind_speed','square_feet']]
y_test = testdf['meter_reading']

In [0]:
print('Size of x_test data', x_test.shape)
print('Size of y_test data', y_test.shape)

Size of x_test data (4025121, 9)
Size of y_test data (4025121,)


In [0]:
x_test.head()

,meter,building_id,primary_use,month,day,air_temperature,dew_temperature,wind_speed,square_feet
12149165,1,959,0,12,12,12.2,8.3,0.0,128536
7366675,0,728,0,2,2,4.0,3.0,8.2,30300
16841290,2,1299,0,5,5,24.4,2.8,2.1,25401
1246907,0,125,0,4,4,5.1,3.1,4.1,16802
19061602,1,1358,2,6,6,20.0,7.2,4.1,61439


In [0]:
y_test.head()

12149165    266.9990
7366675       5.3000
16841290    333.5230
1246907     134.9000
19061602     36.6751
Name: meter_reading, dtype: float64

In [0]:
# Store data into a pickle file to use in the modeling phase
import pickle
with open('clean_data.pickle', 'wb') as f:
    pickle.dump([x_train, y_train, x_test,y_test], f)

# **Setting the model**

Load the pickle file

In [0]:
import pickle
with open('clean_data.pickle', 'rb') as f:
    [x_train,y_train, x_test, y_test] = pickle.load(f)

In [0]:
print('Size of x_train data', x_train.shape)
print('Size of y_train data', y_train.shape)
print('Size of x_test data', x_test.shape)
print('Size of y_test data', y_test.shape)

Size of x_train data (16100484, 9)
Size of y_train data (16100484,)
Size of x_test data (4025121, 9)
Size of y_test data (4025121,)


In [0]:
from sklearn import preprocessing
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, LSTM, GRU, Dropout, BatchNormalization
from keras.models import Sequential
from keras.optimizers import RMSprop,Adam
from keras import regularizers

Using TensorFlow backend.


In [0]:
#RMSE Error
def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true)))

## **Model: Fully connected with 5 layers**

In [0]:
model = Sequential()

# First and second layers
model.add(Dense(500, input_dim=9, activation= "relu"))
# Desactivate 20% of neurons 
model.add(Dropout(0.2))

# Add a 3rd layer
model.add(Dense(100, activation= "relu"))
model.add(Dropout(0.2))

# Add a 4th layer
model.add(Dense(50, activation= "relu"))

#Add the last layer
model.add(Dense(1))

In [0]:
def root_mean_squared_error(y_true, y_pred):
  return K.sqrt(K.mean(K.square(y_pred - y_true)))


metrics = root_mean_squared_error
model.compile(loss='mse', optimizer="rmsprop" ,metrics=[metrics])

In [0]:
 #Normalize the distribution of data

y_train = np.log1p(y_train)


In [0]:
#Prevent OVERFITTING with EarlyStopping Function

es = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=True, mode='auto')

In [0]:
y_train.head()

9606115     3.793616
184779      7.239890
10846520    5.743324
14979167    7.522000
1075753     6.962490
Name: meter_reading, dtype: float64

In [0]:
#assert not np.any(np.isnan(x))

## **Fitting the model**

In [0]:
model.fit(x_train,y_train,epochs=30,batch_size=500,verbose=1,validation_data=(x_test, y_test),callbacks=[es])





Train on 16100484 samples, validate on 4025121 samples
Epoch 1/30





16100484/16100484 [==============================] - 330s 20us/step - loss: 10858.0277 - root_mean_squared_error: 5.7523 - val_loss: 23225920547.3603 - val_root_mean_squared_error: 38327.6023
Epoch 2/30
16100484/16100484 [==============================] - 334s 21us/step - loss: 5.7782 - root_mean_squared_error: 2.1558 - val_loss: 23225920147.9260 - val_root_mean_squared_error: 38327.5811
Epoch 3/30
16100484/16100484 [==============================] - 330s 20us/step - loss: 4.3862 - root_mean_squared_error: 2.0650 - val_loss: 23225918183.6139 - val_root_mean_squared_error: 38327.5418
Epoch 4/30
16100484/16100484 [==============================] - 325s 20us/step - loss: 3.7647 - root_mean_squared_error: 1.9361 - val_loss: 23225920226.2218 - val_root_mean_squared_error: 38327.6114
Epoch 5/30
16100484/16100484 [==============================] - 329s 20us/step - loss: 3.6852 - root_mean_squared_error: 1.9164 - val_los

In [0]:
from keras.models import load_model
model.save('model1.h5')  # creates a HDF5 file 'my_model.h5'

In [0]:
model.load_weights("model1.h5")

## **Validation**

In [0]:
y_test = np.log1p(y_test)

In [0]:
y_predicted= model.predict(x_test ,batch_size=1024, verbose=1)


4025121/4025121 [==============================] - 11s 3us/step


In [0]:
y_predicted[5555]

array([3.6776776], dtype=float32)

In [0]:
y_test = np.log1p(y_test)

In [0]:
from sklearn.metrics import mean_squared_error



In [0]:
print(np.sqrt(mean_squared_error(y_test,y_predicted)))


3.4388107471009532


# **Evaluating the model with testdata**

In [0]:
test_main = pd.read_csv ('test.csv.zip', compression='zip')
test_main.shape

(41697600, 4)

In [0]:
buildings_test = pd.read_csv ('building_metadata.csv' )

buildings_test = buildings_test.drop(columns=['year_built','floor_count'], axis=1)

l=["Religious worship","Utility","Food sales and service","Technology/science","Services","Retail","Manufacturing/industrial","Warehouse/storage","Parking","Healthcare","Other"]
buildings_test.loc[buildings_test.primary_use.isin(l),'primary_use'] = 'others'

buildings_test["primary_use"] = buildings_test["primary_use"].astype('category')
buildings_test["primary_use"] = buildings_test["primary_use"].cat.codes

In [0]:
weather_test=pd.read_csv('weather_test.csv.zip', compression='zip')


In [0]:
weather_test=pd.read_csv('weather_test.csv.zip', compression='zip')
weather_test.shape
weather_test = weather_test.drop(columns=['precip_depth_1_hr','wind_direction','sea_level_pressure','cloud_coverage'], axis=1)


weather_test['wind_speed'].fillna(weather_test['wind_speed'].mean(), inplace=True)
weather_test['air_temperature'].fillna(weather_test['air_temperature'].mean(), inplace=True)
weather_test['dew_temperature'].fillna(weather_test['dew_temperature'].mean(), inplace=True)



In [0]:
data_test =  pd.merge(test_main,buildings_test, on=['building_id'],sort=False)


In [0]:
data_test = data_test.merge(weather_test,left_on=['site_id','timestamp'],right_on=['site_id','timestamp'],how='left')

In [0]:
data_test["month"] = data_test["timestamp"].map(lambda row: row.split(" ")[0].split("-")[1])
data_test["day"] = data_test["timestamp"].map(lambda row: row.split(" ")[0].split("-")[2])
#data_test["time"] = data_test["timestamp"].map(lambda row: row.split(" ")[1].split(":")[0])

data_test['month'] = pd.to_numeric(data_test.month, errors='coerce')
#data_test['time'] = pd.to_numeric(data_test.time, errors='coerce')
data_test['day'] = pd.to_numeric(data_test.month, errors='coerce')

In [0]:
data_test.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,month,day
0,0,0,0,2017-01-01 00:00:00,0,0,7432,17.8,11.7,3.6,1,1
1,129,0,0,2017-01-01 01:00:00,0,0,7432,17.8,12.8,3.1,1,1
2,258,0,0,2017-01-01 02:00:00,0,0,7432,16.1,12.8,3.1,1,1
3,387,0,0,2017-01-01 03:00:00,0,0,7432,17.2,13.3,3.1,1,1
4,516,0,0,2017-01-01 04:00:00,0,0,7432,16.7,13.3,2.6,1,1


In [0]:
data_test = data_test.drop(columns=[ 'row_id','timestamp','site_id'], axis=1)


In [0]:
predicted=model.predict(data_test, batch_size=1024, verbose=1)

41697600/41697600 [==============================] - 108s 3us/step


In [0]:
data_test.head()

,building_id,meter,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,month,day
0,0,0,0,7432,17.8,11.7,3.6,1,1
1,0,0,0,7432,17.8,12.8,3.1,1,1
2,0,0,0,7432,16.1,12.8,3.1,1,1
3,0,0,0,7432,17.2,13.3,3.1,1,1
4,0,0,0,7432,16.7,13.3,2.6,1,1


## **Submission**

In [0]:
#ids = data_test.row_id


In [0]:
len(predicted)

41697600

In [0]:
predicted[5599995]

array([4.1496277], dtype=float32)

In [0]:
submit = pd.read_csv('sample_submission.csv.zip', compression='zip')
predicted = np.expm1(predicted)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in expm1
  


In [0]:
!ls

building_metadata.csv  sample_data		  weather_test.csv.zip
clean_data.pickle      sample_submission.csv.zip  weather_train.csv.zip
model1.h5	       test.csv.zip
my-submission.csv      train.csv.zip


In [0]:
submit['meter_reading'] = predicted

In [0]:
export_csv = submit.to_csv('./my-submission.csv', index=False)


In [0]:
submit.head()

,row_id,meter_reading
0,0,62.410389
1,1,62.410389
2,2,62.410389
3,3,62.410389
4,4,62.410389


In [0]:
!kaggle competitions submit ashrae-energy-prediction -f my-submission.csv -m 'Final submission'

100% 696M/696M [00:13<00:00, 55.8MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III